In [1]:
import numpy as np
import pandas as pd
import pystan
from scipy.stats import norm, multivariate_normal, invwishart, invgamma
from statsmodels.tsa.stattools import acf
import datetime
import sys
import os

from codebase.plot_static import * 
from codebase.file_utils import save_obj, load_obj
from codebase.post_process import * 

%matplotlib inline

%load_ext autoreload
%autoreload 2

/Users/itemgmt/miniconda3/envs/factor-analysis/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [2]:
df = pd.read_csv("../dat/muthen_men.csv")
df.head()

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15
0,4,4,6,7,7,6,7,7,7,7,4,7,7,3,7
1,3,4,6,6,5,5,3,5,5,6,7,5,5,6,6
2,2,3,4,5,5,3,3,3,2,6,3,4,5,6,4
3,7,6,6,5,5,5,5,5,3,3,3,3,5,6,6
4,6,6,5,5,6,6,4,4,4,7,5,6,3,4,4


In [3]:
data = dict()
data['N'] = df.shape[0]
data['K'] = 5
data['J'] = df.shape[1]
data['y'] = df.values

In [4]:
stan_data = dict(N = data['N'], K = data['K'], J = data['J'], yy = data['y'])

In [5]:
with open('./codebase/stan_code/cont/CFA/marg_m.stan', 'r') as file:
    model_code = file.read()
print(model_code)

data {
  int<lower=1> N;
  int<lower=1> K;
  int<lower=1> J;
  matrix[N,J] yy;
}

transformed data{
  real<lower=0> c = 1;
  vector[J] zeros = rep_vector(0, J);
  cov_matrix[J] I_c = diag_matrix(rep_vector(c, J));
}

parameters {
  vector<lower=0>[J] sigma;
  vector<lower=0>[K] sigma_z;
  vector[J] alpha;
  matrix[2,K] beta_free; // 2 free eleements per factor
  matrix[J-3,K] beta_zeros; // 3 zero elements per factor
  cholesky_factor_corr[K] V_corr_chol;
  matrix[N,J] uu;
  cov_matrix[J] Sigma_u;
}

transformed parameters{
  cov_matrix[J] Sigma_epsilon;
  matrix[J,K] beta;
  cov_matrix [K] V ;
  cov_matrix[J] Omega;
  
  Sigma_epsilon = diag_matrix(square(sigma));
  V = multiply_lower_tri_self_transpose(diag_pre_multiply(sigma_z, V_corr_chol));

  for(j in 1:J) {
    for (k in 1:K) beta[j,k] = 0;
  }
  
  // set ones
  for (k in 1:K) beta[1+3*(k-1), k] = 1;

  // set the free elements
  for (k in 1:K) beta[2+3*(k-1) : 3+3*(k-1), k] = beta_free[1:2,k];

  // set the zero elements
  bet

In [ ]:
sm = pystan.StanModel(model_code=model_code, verbose=False)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_705c682db9fe8c2e3bcc88ea4dbc7ba6 NOW.


In [47]:
nowstr = datetime.datetime.now().strftime('%Y%m%d_%H%M%S_') # ISO 8601 format
task_id = 'CFA_NN_marg_muthen_men'
log_dir =  "./log/"+nowstr+"%s/" % task_id

In [48]:
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [12]:
fit_opt = sm.optimizing(data=stan_data)


In [6]:
num_chains = 1
num_samples = 1000
num_warmup = 1000
num_iter = num_samples + num_warmup

In [50]:
fit_run = sm.sampling(data=stan_data, iter=num_iter, warmup=num_warmup, chains=num_chains)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [46]:
# K = 5
# J=15
# for k in range(2,K):
#     print("beta[ %d:%d, %d]="%(1, 3*(k-1), k))
#     print("beta_zeros [ %d:%d, %d]"%(1,3*(k-1), k))
#     print("beta[ %d:%d, %d]="%(4+3*(k-1),J, k))
#     print("beta_zeros [ %d:%d, %d]"%(3*(k-1)+1,J-3, k))
# #     beta[4+3*(k-1):J, k] = beta_zeros[3*(k-1)+1:J-3, k];


In [51]:
save_obj(sm, 'sm', log_dir)
save_obj(fit_run, 'fit', log_dir)
fit=fit_run

/Users/itemgmt/myPhD/factor-analysis/src/codebase/file_utils.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [52]:
print(log_dir)

./log/20190424_231347_CFA_NN_marg_muthen_men/


In [7]:
log_dir = "./log/20190424_231347_CFA_NN_marg_muthen_men/"
sm = load_obj('sm', log_dir)
fit = load_obj('fit', log_dir)



In [8]:
param_names = ['Omega_beta', 'beta', 'V_corr', 'V' , 'uu', 'alpha', 'sigma', 'sigma_z']

stan_samples= fit.extract(permuted=False, pars=param_names)  # return a dictionary of arrays

if num_chains ==1:
    ps = dict()
    for name in param_names:
        ps[name] = np.squeeze(stan_samples[name])
else: 
    ps = stan_samples

In [9]:
np.round(ps['beta'][180], 2)

array([[ 1.  , -0.09, -0.09, -0.11, -0.13],
       [ 0.87,  0.07, -0.  ,  0.1 ,  0.04],
       [ 1.79,  0.05,  0.01,  0.12,  0.3 ],
       [ 0.09,  1.  ,  0.23,  0.12,  0.08],
       [-0.03,  0.71, -0.05, -0.32, -0.1 ],
       [-0.01,  2.47,  0.04, -0.03,  0.08],
       [ 0.1 , -0.05,  1.  ,  0.16, -0.18],
       [-0.09, -0.07,  0.74, -0.15,  0.06],
       [-0.19,  0.01,  0.66, -0.22,  0.03],
       [ 0.14, -0.14,  0.06,  1.  ,  0.03],
       [ 0.07,  0.04, -0.15,  0.79, -0.19],
       [-0.11,  0.02,  0.08,  0.5 ,  0.  ],
       [-0.  ,  0.01, -0.06, -0.02,  1.  ],
       [ 0.01, -0.04, -0.05,  0.12,  0.63],
       [ 0.17,  0.12,  0.1 , -0.02,  1.06]])

In [11]:
mb = np.zeros((data['J'], data['K']))
mb[:3,0] = np.array([0.842, 0.394, 0.479 ])
mb[3:6,1] = np.array([0.683, 0.078, 0.579 ])
mb[6:9,2] = np.array([0.748, 0.754, 0.575])
mb[9:12,3] = np.array([0.801, 0.708, 0.613])
mb[12:,4] = np.array([0.732, 0.672, 0.651])


In [12]:
muthen_results = dict()
muthen_results['beta'] = mb


In [13]:
our_results=dict()
our_results['beta'] = np.average(ps['beta'], axis=0)

In [14]:
muthen_results['beta'][:,0]

array([0.842, 0.394, 0.479, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   ])

In [15]:
for i in range(5):
    print("%d,%d= %.1f"%(3*i,i,ps['beta'][0,3*i,i]))

0,0= 1.0
3,1= 1.0
6,2= 1.0
9,3= 1.0
12,4= 1.0


In [22]:
# %%opts Curve {+axiswise} [width=600, height=200, ] 
plots = []
# for i in range(data['J']):
#     for j in range(data['K']):
for i in range(2):
    for j in range(2):
            plots.append(plot_trace(ps['beta'][:,i,j],
                     true_value=None,
                     title = 'Posterior distribution for beta(%s,%s)'%(i,j)).options(fig_inches=8, aspect=3))
layout = hv.Layout(plots).options(tight=False)
layout.cols(1)

:Layout
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Posterior_distribution_for_beta_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)

In [24]:
plots = []
# for i in range(data['J']):
#     for j in range(data['K']):
for i in range(2):
    for j in range(2):
            plots.append(plot_trace(acf(ps['Omega_beta'][:,i,j]),
                     title = 'Autocorrelation of Omega(%s,%s)'%(i,j)).options(fig_inches=8, aspect=3))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_0_comma_1_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_0_right_parenthesis :Curve   [x]   (y)
   .Curve.Autocorrelation_of_Omega_left_parenthesis_1_comma_1_right_parenthesis :Curve   [x]   (y)

In [25]:
def get_residuals(ps_u, by_axis, absval = True, sort=False):
    """
    params
    ps_u posterior samples of u
    by_axis 0=residuals per iterm, 1=residuals per subject
    """
    mean_u = np.mean(ps_u, axis=0) #mean posterior residual matrix
    std_u = np.sqrt(np.mean(ps_u**2, axis=0)- mean_u**2) #std of posterior residual matrix
    if absval:
        res = pd.DataFrame(np.sum(np.abs(mean_u/std_u), axis=by_axis)).reset_index()
    else:
        res = pd.DataFrame(np.sum(mean_u/std_u, axis=by_axis)).reset_index()
    res.columns = ['index', 'residual']
    
    if sort:
        res.sort_values('residual', ascending=False, inplace=True)
    return res

get_residuals(ps['uu'], 1, True, True)


,index,residual
160,160,47.277394
414,414,45.054608
241,241,43.036886
168,168,29.498254
225,225,28.600894
90,90,28.137773
92,92,23.254118
445,445,23.177498
359,359,23.160026
284,284,23.009538


In [27]:
# %%opts Bars {+axiswise} [width=1000, height=300, ] 
res = get_residuals(ps['uu'], 1, True, True)

hv.Bars(res[:20], hv.Dimension('index'), 'residual',\
        label='Top 20 Residuals').options(color='blue', xrotation=90).options(fig_inches=8, aspect=3)


:Bars   [index]   (residual)